In [ ]:
import columns
import paths
from utils import split_train_test, downsample_data

# Fire occurrence model (classification)


In [ ]:
from catboost import CatBoostClassifier
import pandas as pd

df = pd.read_csv(paths.inmet_inpe_linked_file)
df = downsample_data(df)

Xc = df.drop(columns=[columns.fire_intensity_column, columns.fire_occurrence_column])
yc = df[columns.fire_occurrence_column]

Xc_train, Xc_test, yc_train, yc_test = split_train_test(Xc, yc, stratify=yc)

model_cls = CatBoostClassifier(
    loss_function='Logloss',
    eval_metric='AUC',
    custom_metric=['Accuracy', 'Precision', 'Recall', 'F1'],
    learning_rate=0.1,
    depth=8,
    iterations=150,
    od_type='Iter',  # early stopping type
    od_wait=10,  # patience for early stopping
    use_best_model=True,
    verbose=True,
    train_dir=paths.classification_info_dir
)

model_cls.fit(Xc_train, yc_train, eval_set=(Xc_test, yc_test))
model_cls.save_model(paths.classifier_cbm_file)

# Fire intensity model (regression)

In [ ]:
from catboost import CatBoostRegressor
import pandas as pd

df = pd.read_csv(paths.inmet_inpe_linked_file)
df.dropna(subset=[columns.fire_intensity_column], inplace=True)  # Remove rows with missing target values

Xr = df.drop(columns=[columns.fire_intensity_column, columns.fire_occurrence_column])
yr = df[columns.fire_intensity_column]

Xr_train, Xr_test, yr_train, yr_test = split_train_test(Xr, yr)

model_reg = CatBoostRegressor(
    loss_function='RMSE',
    eval_metric='RMSE',
    custom_metric=['RMSE', 'MAE', 'R2', 'MAPE'],
    learning_rate=0.1,
    depth=8,
    iterations=1000,
    od_type='Iter',  # early stopping type
    od_wait=20,  # patience for early stopping
    use_best_model=True,
    verbose=True,
    train_dir=paths.regression_info_dir
)

model_reg.fit(Xr_train, yr_train, eval_set=(Xr_test, yr_test))
model_reg.save_model(paths.regressor_cbm_file)